In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import re
import os
from pathlib import Path
from functools import partial
from itertools import chain
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from api.utils import parse_offset, today, get_next_n
from api.data.base import data_path, radar_path
from api.gui.base import View
from api.gui.params import curves, bbg_params
from api.blpw import BlpQuery
from api.gui.radar import RepeatedTimer

pd.options.plotting.backend = "plotly"

In [3]:
bq = BlpQuery(timeout=50000).start()

dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'
bbg_options = {
    'calendarCodeOverride': '5D',
    "nonTradingDayFillOption": "ALL_CALENDAR_DAYS",
    "nonTradingDayFillMethod": "PREVIOUS_VALUE"
}

In [4]:
curves_df = pd.DataFrame(curves).set_index('ccy').loc[['EUR', 'USD', 'GBP']]

In [5]:
class LiveCurvesLocal:
    def __init__(self, date=None, curves=None):
        self.date = date or pd.Timestamp.today().floor('d')
        self.curves = curves
        try:
            self.sheet_mkt = bh.bhCreateSheetMarket(
                _handle='sheet_mkt',
                _mkt='LIVE',
                _asof=self.date,
                _mktdefs=mkt_defs,
                _calprov=mx_cals,
                _mtgdts=mtg_dts,
                _crvdbsvr="SQLP1-A",
                _crvdbname="Curves",
                _mktdatasvr="localhost",
                _mktdataport=60000,
                _snapdbsvr="SQLP1-B",
                _snapdbname="Snap",
            )
        except TypeError:
            self.sheet_mkt = bh.bhCreateSheetMarket(
                _hdl='sheet_mkt',
                _mkt='LIVE',
                _asof=self.date,
                _mktdefs=mkt_defs,
                _calprov=mx_cals,
                _mtgdts=mtg_dts,
                _crvdbsvr="SQLP1-A",
                _crvdbnm="Curves",
                _mdssvr="localhost",
                _mdsport=60000,
                _snapdbsvr="SQLP1-B",
                _snapdbnm="Snap",
            )
    
    @property
    def mkt_snap(self):
        return bh.bhCurveMarketSnap(
            _handle='crv_mkt_snap',
            _market=self.sheet_mkt,
            _curves=self.curves
        )
    
    @property
    def mkt_snap_db(self):
        return bh.bhCurveMarketSnapAsDb(
            _handle='crv_mkt_snap_db',
            _marketsnap=self.mkt_snap
        )
    
    @property
    def mkt_snap_db_curves(self):
        return bh.bhDbLookUp(
            _rsid=self.mkt_snap_db,
            _lookupvalues="LIVE",
            _lookupcols="name",
            _returncols="curves"
        )
    
    @property
    def db_curves(self):
        return self.mkt_snap_db_curves
    

In [6]:
class COBCurvesRemote:
    def __init__(self, date=None, curves=None, offset=None):
        self.date = date or pd.Timestamp.today().floor('d') - BDay()
        if offset:
            self.date = pd.Timestamp.today().floor('d') - parse_offset(offset)
        self.curves = curves
        
        self.db_mkt = bh.bhCreateCurvesDbMarket(
            _handle=f'crv_db_mkt_{self.date}',
            _mktname="COB",
            _asofdate=self.date,
            _datadate=self.date,
            _mktdefs=mkt_defs,
            _cals=mx_cals,
            _curvesserver="SQLP1-A",
            _curvesdb="Curves",
            _minextraptnr='80y'
        )
        
        self.mkt_snap_cob = bh.bhMarketSnap(
            _handle=f'mkt_snap_cob_{self.date:%Y%m%d}',
            _market=self.db_mkt,
            _curves=self.curves
        )
        
        self.db_curves = self.mkt_snap_cob

In [7]:
def get_sw_rate_from_ccy(
    ccy,
    tenor,
    db_curves,
    curves_df
):
    try:
        curves_df = curves_df.set_axis(curves_df.index.str.lower())
        disc_ = bh.bhDbLookUp(
            _rsid=db_curves, 
            _lookupvalues=curves_df.loc[ccy.lower(), 'disc'],
            _lookupcols="name",
            _returncols="curve",
            _matchtype=0
        )

        proj_ = bh.bhDbLookUp(
            _rsid=db_curves, 
            _lookupvalues=curves_df.loc[ccy.lower(), 'std'],
            _lookupcols="name",
            _returncols="curve",
            _matchtype=0
        )

        rate = bh.bhRate2(
            _crv=disc_,
            _mtyortenor=tenor,
            _type='sw',
            _name=curves_df.loc[ccy.lower(), 'std'],
            _mktdefs=mkt_defs,
            _calprov=mx_cals,
            _leavemtyflag=1,
            _mul=100,
            _projcurve=proj_,
            _mtgdates=mtg_dts
        )
    except Exception as e:
        print(e, ccy, tenor)
        return np.nan
    return rate

In [8]:
def get_rate_from_generator(
    generator, 
    efforfwd, 
    mtyortenor,
    db_curves,
    collateral='LCH',
    
):
    rate = bh.bhFiRate(
        _efforfwd=efforfwd,
        _mtyortenor=mtyortenor,
        _generatorname=generator,
        _curves=db_curves,
        _collateral=collateral,
        _generators=gens
    )
    return rate


def get_rates_from_generator(
    generator, 
    efforfwd, 
    mtyortenor,
    db_curves,
    collateral='LCH',
    drop_duplicates=True,
    
):
    if drop_duplicates:
        efforfwd = pd.Series(efforfwd).drop_duplicates().to_list()
        mtyortenor = pd.Series(mtyortenor).drop_duplicates().to_list()
        
    res_db = bh.bhFiRates(
        _handle=uuid4().hex,
        _fwdterms=efforfwd,
        _tenors=mtyortenor,
        _generatorname=generator,
        _curves=db_curves,
        _collateral=collateral,
        _generators=gens
    )
    df = pd.DataFrame(bh.bhDbGetValues(_rsid=res_db, _columnheadings=True))
    return df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True).infer_objects()

In [9]:
live_curves = LiveCurvesLocal(curves=curves_df.melt()['value'].drop_duplicates().dropna().to_list())
cob_curves = COBCurvesRemote(curves=curves_df.melt()['value'].drop_duplicates().dropna().to_list())
# live_curves = LiveCurvesLocal()
# cob_curves = COBCurvesRemote()

In [10]:
spot_tenors = [
    (0, 'o/n'),
    (0, '1m'),
    (0, '3m'),
    (0, '6m'),
    (0, '1y'),
    (0, '2y'),
    (0, '3y'),
    (0, '4y'),
    (0, '5y'),
    (0, '6y'),
    (0, '7y'),
    (0, '8y'),
    (0, '9y'),
    (0, '10y'),
    (0, '12y'),
    (0, '15y'),
    (0, '20y'),
    (0, '25y'),
    (0, '30y'),
    (0, '40y'),
    (0, '50y'),

]

fwd_tenors = [
    (0, '1y'),
    ('1y', '1y'),
    ('2y', '1y'),
    ('3y', '1y'),
    ('4y', '1y'),
    ('5y', '1y'),
    ('6y', '1y'),
    ('7y', '1y'),
    ('8y', '1y'),
    ('9y', '1y'),
    ('10y', '2y'),
    ('12y', '3y'),
    ('15y', '5y'),
    ('20y', '5y'),
    ('25y', '5y'),
]

spot_tenors_front_end = spot_tenors[:6]
spot_tenors_medium_long_end = spot_tenors[4:]

In [11]:
spot_params = [
    {
        'gen': 'USD SOFR',
        'tenors': spot_tenors
    },
    {
        'gen': 'EUR EONIA',
        'tenors': spot_tenors_front_end
    },
    {
        'gen': 'EURIBOR A 6M*',
        'tenors': spot_tenors
    },
    {
        'gen': 'GBP SONIA',
        'tenors': spot_tenors
    }
]

In [12]:
fwd_params = [
    {
        'gen': 'USD SOFR',
        'tenors': fwd_tenors
    },
    {
        'gen': 'EURIBOR A 6M*',
        'tenors': fwd_tenors
    },
    {
        'gen': 'GBP SONIA',
        'tenors': fwd_tenors
    }
]

In [13]:
class SpotForward:
    def __init__(self, spot_params, fwd_params):
        self.spot_params = spot_params
        self.fwd_params = fwd_params
        
        self.params = {
            'spot': spot_params,
            'fwd': fwd_params
        }
        
    def get_data(self, db_curves=None, **kwargs):
        if isinstance(db_curves, dict):
            res = {}
            for curve_type, curves in db_curves.items():
                res[curve_type] = self._get_data(db_curves=curves)
                
            try:
                res['live']['spot']['change'] = res['live']['spot']['rate'] - res['cob']['spot']['rate']
                res['live']['fwd']['change'] = res['live']['fwd']['rate'] - res['cob']['fwd']['rate']
            except Exception as e:
                raise e
        elif isinstance(db_curves, list):
            res = []
            for curves in db_curves:
                res.append(self._get_data(db_curves=curves))
        else:
            res = self._get_data(db_curves=db_curves)
        self.data = res 
        return res
    
    def _get_data(self, db_curves=None, **kwargs):
        rates = {}
        for key, params in self.params.items():
            rates_data = {}
            for params_ in params:
                starts, tenors = [*zip(*params_['tenors'])]
                rates_df = get_rates_from_generator(
                    generator=params_['gen'],
                    efforfwd=starts,
                    mtyortenor=tenors,
                    db_curves=db_curves
                )
                rates_data[params_['gen']] = rates_df
            rates_data = pd.concat(rates_data)
            rates[key] = rates_data
        return rates

In [14]:
class Futures:
    _base_tickers = {
        'EUR': 'ER',
        'USD': 'SFR',
        'GBP': 'SFI'
    }
    _base_spot_tickers = {
        'EUR': 'EUR003M Index',
        'USD': 'SOFRRATE Index',
        'GBP': 'SONIO/N Index'
    }
    
    def __init__(self, base_tickers=None, n=8):
        self.base_tickers = base_tickers or self._base_tickers
        self.n = n
    
    def get_tickers(self, return_dict=False):
        exps = get_next_n(self.n)
        base_tickers = self.base_tickers
        res = {ccy: [self._base_spot_tickers[ccy], *[f"{base_tickers[ccy]}{exp} Comdty" for exp in exps]] for ccy in base_tickers}
        return res if return_dict else [*chain.from_iterable(res.values())]

    def get_data(self, db_curves=None, cob_date=None, **kwargs):
        cob_date = cob_date or today() - BDay()
        tickers = self.get_tickers()
        fields = ['px_last', 'security_des']
        live = bq.bdp(
            tickers,
            fields
        ).rename({'px_last': 'live'}, axis=1).set_index('security')

        cob = bq.bdh(
            tickers,
            fields[:1],
            start_date=f"{cob_date:{bbg_dt_fmt}}",
            end_date=f"{cob_date:{bbg_dt_fmt}}",
            options=bbg_options,
        ).rename({'px_last': 'cob'}, axis=1).set_index('security')
        data = pd.concat([live, cob], axis=1)
        data['change'] = data['live'] - data['cob']
        self.data = data
        return data

In [15]:
class Market:
    _params = pd.DataFrame(bbg_params)
    
    def __init__(self, params=None):
        self.params = params or self._params
    
    def get_tickers(self):
        return self.params['ticker']
        
    def get_data(self, db_curves=None, cob_date=None, **kwargs):
        cob_date = cob_date or today() - BDay()
        fields = ['px_last', 'security_des']
        tickers = self.get_tickers()
        live = bq.bdp(
            tickers,
            fields
        ).rename({'px_last': 'live'}, axis=1).set_index('security')

        cob = bq.bdh(
            tickers,
            fields[:1],
            start_date=f"{cob_date:{bbg_dt_fmt}}",
            end_date=f"{cob_date:{bbg_dt_fmt}}",
            options=bbg_options,
        ).rename({'px_last': 'cob'}, axis=1).set_index('security')
        self.live = live
        self.cob = cob
        data = pd.concat([live, cob], axis=1)
        data['change'] = data['live'] - data['cob']

        to_add = self.params.set_index('ticker').rename_axis('security')
        data = pd.concat([data, to_add], axis=1)
        self.data = data
        return data

In [16]:
class DataCollector:
    def __init__(self, curves=None, collectors=None):
        self.live_curves = LiveCurvesLocal(curves=curves)
        self.cob_curves = COBCurvesRemote(curves=curves)
        self.collectors = collectors
        self._curves = curves
        self._offset = None
        
    @property
    def offset(self):
        return self._offset
    
    @offset.setter
    def offset(self, value):
        self._offset = value
        self.cob_curves = COBCurvesRemote(curves=self._curves, offset=value)
    
    
    def get_data(self, update=True, **kwargs):
        if not hasattr(self, 'db_curve') or update:
            db_curves ={
                'live': self.live_curves.db_curves,
                'cob': self.cob_curves.db_curves
            }
            self.db_curve = db_curves
        db_curve = self.db_curve
        for name, collector in self.collectors.items():
            collector.get_data(db_curves=db_curves, cob_date=self.cob_curves.date)
            
    def __getitem__(self, item):
        return self.collectors[item]

In [17]:
def default_styler(
    df, 
    precision=4, 
    date_format='%Y-%m-%d', 
    na_rep='-', 
    hover_color="#ffff99", 
    formatters=None, 
    multipliers=None,
    apply=None,
    applymap=None,
):

    th_props = [
        ('font-size', '13px'),
        ('text-align', 'center'),
        ('font-weight', 'bold'),
        # ('color', '#6d6d6d'),
        ('background-color', '#f7f7f9'),
        ('border-bottom', '1px solid  #d7d7d7')
    ]
    
    tr_props = [
        ("line-height", "15px"),
        # ('border-color', 'white'),
        ('background-color', 'white'),
        ('border-bottom', '1px solid  #d7d7d7'),
    ]
    
    selectors = [
        {"selector": "tr", "props": tr_props},
        {"selector": "td, th", "props": "line-height: inherit; padding-top: 3px; padding-bottom: 3px;"},
        {'selector': 'th', 'props': th_props},
        *hover(hover_color)
    ]
    
    if multipliers:
        if isinstance(multipliers, int):
            df = df * mult
        else:
            for mult in multipliers:
                df[mult] = multipliers[mult] * df[mult]
    
    date_cols = {}
    if date_format:
        date_format = f'{{:{date_format}}}'
        for col in df:
            if np.issubdtype(df[col].dtype, np.datetime64):
                date_cols[col] = partial(format_time_nat, fmt=date_format) 
    
    formatters = {**(formatters or {}), **date_cols}
    
    styler = df.style.\
                set_table_styles(selectors).\
                format(
        formatter=formatters,
        precision=precision,
        na_rep=na_rep,
    )
    
    if apply and not isinstance(apply, list):
        apply = [apply]
        for fn in apply:
            if not isinstance(fn, dict):
                fn = {'dict': fn}
            styler = styler.apply(**fn)
            
    if applymap and not isinstance(applymap, list):
        applymap = [applymap]
        for fn in applymap:
            if not isinstance(fn, dict):
                fn = {'dict': fn}
            styler = styler.applymap(**fn)
    
    return styler


def negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color


def format_time_nat(t, fmt='{:%Y-%m-%d}'):
    try:
        return fmt.format(t) # or strftime
    except ValueError:
        return t
    
    
def hover(hover_color="#ffff99"):
    return [
        dict(
            selector="tr:hover",
            props=[("background-color", "%s" % hover_color)]
        ),
        dict(
            selector="th:hover",
            props=[("background-color", "%s" % hover_color)]
        ),
    ]


class DFOutput(View):
    def __init__(self, df, out=None, style_fn=default_styler, styler_kwargs=None, **kwargs):
        self._df = df
        self.out = out or w.Output()
        self.style_fn = style_fn or (lambda df: df.style)
        self.styler_kwargs = styler_kwargs or {}
        super().__init__(**kwargs)
        
    def make_widgets(self, **kwargs):
        with self.out:
            self.out.clear_output(wait=True)
            # display(self.style_fn(self._df, **self.styler_kwargs))
            self.out.append_display_data(self.style_fn(self._df, **self.styler_kwargs))
        
    def make_view(self, **kwargs):
        self.view = self.out
    
    @property
    def df(self):
        return self._df

    
    @df.setter
    def df(self, value):
        self._df = value
        with self.out:
            self.out.clear_output()
            display(self.style_fn(self._df, **self.styler_kwargs))
            # self.out.append_display_data(self.style_fn(self._df, **self.styler_kwargs))

In [18]:
dc = DataCollector(
    curves=curves_df.melt()['value'].drop_duplicates().dropna().to_list(),
    collectors={
        'sf': SpotForward(spot_params=spot_params, fwd_params=fwd_params),
        'fut': Futures(),
        'mkt': Market()
    }
)
dc.get_data()

In [19]:
class SpotTable:
    def __init__(self, gen):
        self.gen = gen
        self.out = w.Output()
    
    def get_table(self, data, gen):
        return data['live']['spot'].loc[gen].set_index('in.mty')[['rate', 'change']].rename_axis(None)
    
    def get_widget(self, dc):
        gen = self.gen
        widget = DFOutput(
            df=self.get_table(dc['sf'].data, gen).rename({'rate': gen}, axis=1),
            #out=self.out,
            styler_kwargs=dict(
                 formatters={gen: '{:.3%}', 'change': '{:.2f}'},
                 multipliers={'change': 10000},
                 applymap={'func': negative_red, 'subset': 'change'}
             ),
        )
        return widget.view

In [20]:
class FutureTable:
    def __init__(self, gen, ccy):
        self.gen = gen
        self.ccy = ccy
        self.out = w.Output()

    def get_widget(self, dc):
        ccy = self.ccy
        gen = self.gen
        fut_df = dc['fut'].data.loc[dc['fut'].get_tickers(return_dict=True)[ccy], ['live', 'change']].rename({'live': f"{gen} Fut"}, axis=1)
        fut_df = fut_df.set_index(fut_df.index.str.replace(' Comdty', '').str[-2:].str.replace('ex', 'spot')).rename_axis(None)
        fut_df.loc['spot', f"{gen} Fut"] = 100 - fut_df.loc['spot', f"{gen} Fut"]
        fut_df.loc['spot', "change"] = np.nan
        widget = DFOutput(
            df=fut_df,
            #out=self.out,
            styler_kwargs=dict(
                 formatters={f"{gen} Fut": '{:.3f}', 'change': '{:.2f}'},
                 applymap={'func': negative_red, 'subset': 'change'}
             ),
        )
        
        return widget.view

In [21]:
class KeyChart:
    def __init__(self, gen, **kwargs):
        self.gen = gen
        self.kwargs = kwargs
        
    def get_widget(self, dc):
        data = dc['sf'].data
        return self.make_fwd_plot(data, self.gen, **self.kwargs)
    
    def make_fwd_plot(self, data, gen, scale=100, fmt='.2f', **kwargs):
        plot_df = pd.concat(
            [
                data['live']['fwd'].set_index(['in.eff', 'in.mty'] , append=True).droplevel(1).rename_axis(3*[None])['rate'].rename('live'),
                data['cob']['fwd'].set_index(['in.eff', 'in.mty'], append=True).droplevel(1).rename_axis(3*[None])['rate'].rename('cob')
            ], 
            axis=1
        )
        plot_df['change'] = plot_df['live'] - plot_df['cob']
        plot_df = plot_df.loc[[(gen, elm[0], elm[1]) for gen in plot_df.index.get_level_values(0).unique() for elm in fwd_tenors]]
        plot_df = plot_df.set_index(
            pd.MultiIndex.from_arrays(
                [
                    plot_df.index.get_level_values(0),
                    plot_df.index.get_level_values(1).astype(str) + 'x' + plot_df.index.get_level_values(2).astype(str)
                ],
            )
        )
        plot_df_ = plot_df.loc[gen] * scale
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig2 = plot_df_[['live', 'cob']].plot(markers=True)
        fig2.data[0].line.color = 'red'
        fig2.data[1].line.color = 'orange'
        fig3 = plot_df_['change'].plot.bar()
        # fig3.data[0].width = [2] * len(plot_df_)
        # fig3.data[0].offset = 0
        fig3.data[0].marker.line = dict(width=1, color='white')
        fig.add_traces([*fig2.data, *fig3.data], secondary_ys=[True, True, False])\
            .update_yaxes(showgrid=False, tickformat=fmt, nticks=10)\
            .update_xaxes(tickangle=45)\
            .update_layout(
            template='plotly_white', 
            height=kwargs.get('height', 500), 
            title=gen, 
            bargap=0,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )
        return go.FigureWidget(fig)

In [22]:
class GroupMktTalbe:
    def __init__(self, group, pct=True):
        self.group = group
        self.pct = pct
        self.out = w.Output()
            
    def get_table(self, data, group):
        return 
    
    def get_widget(self, dc):
        data = dc['mkt'].data.copy()
        if self.pct:
            data['change'] = data['change'] / data['cob']
            
        group = self.group
        df = data.query(f'group == "{group}"').set_index('display_name')[['live', 'change']].rename({'live': group}, axis=1).rename_axis(None)
        widget = DFOutput(
            df=df,
            #out=self.out,
            styler_kwargs=dict(
                 formatters={group: '{:.2f}', 'change': '{:.2f}' if not self.pct else '{:.2%}'},
                 applymap={'func': negative_red, 'subset': 'change'}
             ),
        )
        return widget.view

In [23]:
import logging

In [24]:
def on_change_update_cob_curves(widget, event, payload, self):
    self.update_cob_curves()
    
    
def on_click_change_timer(widget, event, payload, self):
    self.start_stop_timer()
    

def on_click_refresh(widget, event, payload, self):
    widget.loading = True
    try:
        logging.warning('updating')
        self.refresh_view()
        logging.warning('updated')
    except Exception as e:
        widget.color = 'error'
        raise e
    else:
        widget.color = 'success'
    finally:
        widget.loading = False

In [61]:
class Monitor(View):
    
    def __init__(self, dc, **kwargs):
        self.dc = dc
        super().__init__(**kwargs)
    
    
    def make_widgets(self, **kwargs):
        refresh_btn = v.Btn(
            fab=True,
            # dark=True,
            x_small=True,
            plain=True,
            class_='ma-2',
            children=[v.Icon(children=['mdi-sync'])]
        )
            
        timer_tf = v.TextField(
            v_model=45,
            label='time',
            clearable=True,
            dense=True,
            outlined=True,
            type="number",
            class_="mr-0"

        )

        change_tf = v.TextField(
            v_model='1d',
            label='change',
            clearable=True,
            dense=True,
            outlined=True,
            class_="mr-0"

        )

        timer_btn = v.Btn(
            fab=True,
            # dark=True,
            x_small=True,
            plain=True,
            class_='mt-2 mb-2 mr-2 ml-0',
            children=[v.Icon(children=['mdi-timer'])]
        )
        
        self.refresh_btn = refresh_btn
        self.timer_tf = timer_tf
        self.timer_btn = timer_btn
        self.change_tf = change_tf
        
    
    def make_view(self):
        self.html = w.HTML(self.get_html_text())
        self.param_box = v.Row(
            children=[
                v.Col(
                    cols=1.0,
                    children=[self.timer_tf],
                    class_="my-0 py-0 mr-0 pr-0"
                ),
                v.Col(
                    cols=1.0,
                    children=[self.timer_btn, self.refresh_btn],
                    class_="my-0 py-0"
                ),
                v.Col(
                    cols=1.0,
                    children=[self.change_tf],
                    class_="my-0 py-0 mr-0 pr-0"
                ),
                v.Col(
                    cols=2,
                    children=[self.html],
                    class_="my-0 py-0"
                ),
            ]
        )
        
        self.container = v.Container(class_="ma-0 pa-0")

        self.view = w.VBox([self.param_box, self.container])
    
    def get_html_text(self, time=None):
        if not time:
            return f'<b>last update: -'
        else:
            return f'<b>last update: {pd.Timestamp.today(): %H:%M:%S}'
        
    
#     @property
#     def refresher(self):
#         return self._refresher
    
#     @refresher.setter
#     def refresher(self, value):
#         self.refresh_view2()
        
    def refresh_view(self):
        dc = self.dc
        dc.get_data()
        # self.refresher = True
        
    # def refresh_view2(self):
        # dc = self.dc
        children = [
            v.Row(
                children=[
                    v.Col(
                        cols=2, 
                        children=[
                            w.VBox([usd_spot.get_widget(dc), usd_fut.get_widget(dc)])

                        ]
                    ),
                    v.Col(
                        cols=2, 
                        children=[
                            w.VBox([eur_spot.get_widget(dc),  eur_fut.get_widget(dc), eur_spot2.get_widget(dc)])

                        ]
                    ),

                    v.Col(
                        cols=2, 
                        children=[
                            w.VBox([gbp_spot.get_widget(dc),  gbp_fut.get_widget(dc)])

                        ]
                    ),
                    v.Col(
                        cols=2, 
                        children=[
                            w.VBox([mkt_equity.get_widget(dc),  mkt_fx.get_widget(dc), mkt_comdty.get_widget(dc)])

                        ]
                    ),
                ]
            ),
            v.Row(
                children=[
                    v.Col(cols=4, children=[usd_chart.get_widget(dc)]),
                    v.Col(cols=4, children=[eur_chart.get_widget(dc)]),
                    v.Col(cols=4, children=[gbp_chart.get_widget(dc)]),
                ]
            )
        ]
        self.container.children = children
        self.html.value = self.get_html_text(True)
        
        
    def link(self, **kwargs):
        self.refresh_btn.on_event(
            'click',
            partial(on_click_refresh, self=self)
        )
        self.timer_btn.on_event(
            'click',
            partial(on_click_change_timer, self=self)
        )

        self.change_tf.on_event(
            'change',
            partial(on_change_update_cob_curves, self=self)
        )
        super().link(**kwargs)
        
    def update_cob_curves(self):
        self.dc.offset = self.change_tf.v_model
        
    
    def start_stop_timer(self):
        sec = int(self.timer_tf.v_model)
        try:
            rt = self.rt
            if not rt.is_running:
                rt.interval = sec
        except AttributeError:
            self.rt = RepeatedTimer(self.refresh_view, sec)
            rt = self.rt

        if rt.is_running:
            self.timer_btn.color = None
            rt.stop()
        else:
            self.timer_btn.color = 'success'
            rt.start()

In [62]:
usd_spot = SpotTable(gen='USD SOFR')
usd_fut = FutureTable(gen='USD SOFR', ccy='USD')
usd_chart = KeyChart(gen='USD SOFR', height=300)

In [63]:
eur_spot = SpotTable(gen='EURIBOR A 6M*')
eur_spot2 = SpotTable(gen='EUR EONIA')
eur_fut = FutureTable(gen='EURIBOR A 6M*', ccy='EUR')
eur_chart = KeyChart(gen='EURIBOR A 6M*', height=300)

In [64]:
gbp_spot = SpotTable(gen='GBP SONIA')
gbp_fut = FutureTable(gen='GBP SONIA', ccy='GBP')
gbp_chart = KeyChart(gen='GBP SONIA', height=300)

In [65]:
mkt_equity = GroupMktTalbe(group='equity')
mkt_fx = GroupMktTalbe(group='fx', pct=False)
mkt_comdty = GroupMktTalbe(group='commodity')

In [66]:
m = Monitor(dc)
m

In [ ]:
m.container.children[0].children[0].children[0]

In [37]:
html = w.HTML(f'<b>last update: {pd.Timestamp.today(): %H:%M:%S}')

In [41]:
html.value

'<b>last update:  21:21:27'